# Pasos:

1. Particionar datos de Entrenamiento (80%) y Pruebas(20%) 
2. Entrenar Modelos a evaluar:
    - XGBoost
    - SVM
    - Ensamble Modelo (RandomForest o Arbol de decisión)
    - Naive Bayes - Gausiano
3. Evaluar los hiperparametros (RandomSearchCV)
4. Entrenar modelos con mejores hiperparámetros encontrados en 3.
5. Evaluar modelos con data de pruebas (Accuracy)

## 1) Lectura de datos - Distribución de conjunto de datos

La distribución del conjunto de datos será de la siguiente manera: 

- Entrenamiento (80%)
- Pruebas (20%)

--- Ya fue aplicado en 2. Pre-procesamiento de datos

In [24]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt

from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV

In [38]:
#Lectura del dataset


#Lectura del dataset

X_TRAIN_DF = pd.read_csv('data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv', header=0, sep=",")
X_TEST_DF  = pd.read_csv('data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv', header=0, sep=",")
Y_TRAIN_DF = pd.read_csv('data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv', header=0, sep=",")
Y_TEST_DF  = pd.read_csv('data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv', header=0, sep=",")

In [39]:
var_objetivo_lst = [
    'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
    'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
    'REC_IM','P_IM_STEN'
]

## 2) Entrenar Modelos a evaluar

In [40]:
## Resultados con la métrica de exactitud (Accuracy) de cada uno de los modelos.
resultados_modelo_acc_df = pd.DataFrame(columns=["Modelo"]+var_objetivo_lst)

### 2.1) Random Forest

Entrenamos el modelo de Random Forest de tipo clasificación.

In [47]:
for var_objetivo in var_objetivo_lst:
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_test = X_TEST_DF.loc[X_TEST_DF["var_objetivo"]==var_objetivo].dropna(axis='columns')
    y_test = Y_TEST_DF.loc[Y_TEST_DF["var_objetivo"]==var_objetivo, var_objetivo]
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    X_test.drop(["var_objetivo"], axis=1, inplace=True)
    
    
    rf = RandomForestClassifier(oob_score = True)
    rf.fit(X_train, y_train)

    print('Exactitud del modelo inicial en entrenamiento:', rf.score(X_train, y_train))
    print('Exactitud del modelo inicial en entrenamiento (Out of Bag):', rf.oob_score_)
    print('Exactitud del modelo inicial en validación:', rf.score(X_test, y_test))

    #### Búsqueda aleatoria de mejores hiperparámetros
    # Definición de Grilla
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features = ['auto', 10, 11, 12]  # 'auto' equivale a 'sqrt'; None equivale a todas
    max_depth = [int(x) for x in np.linspace(10, 220, num = 11)] + [None]
    min_samples_leaf = [1, 2, 4]

    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_leaf': min_samples_leaf}

    print('Los valores a probar en la búsqueda aleatoria son:')
    pprint(random_grid)

    print()
    print('Si se probara todas las combinaciones se requeriría entrenar', 
          len(random_grid['n_estimators']) *
          len(random_grid['max_features']) *
          len(random_grid['max_depth']) *
          len(random_grid['min_samples_leaf']),
          'modelos'
          )

    rf = RandomForestClassifier(oob_score=True)
    rf_random = RandomizedSearchCV(estimator = rf, 
                                   param_distributions = random_grid, 
                                   cv = 3,          # Validación cruzada 3-fold
                                   verbose=2, 
                                   random_state=0, 
                                   n_jobs = -1      # Paralelizar en todos los cores disponibles
                                   )
    rf_random.fit(X_train, y_train)

    ## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
    rf_random_best = rf_random.best_estimator_

    print('Los hiperparámetros del mejor modelo son:')
    pprint(rf_random.best_params_)
    print()

    resultados_modelo_acc_df=resultados_modelo_acc_df.append(
        {
            'Modelo' : 'RandomForest' , 
            'Accuracy Train' : rf_random_best.score(X_train, y_train),
            'Accuracy Test' : rf_random_best.score(X_test, y_test)
        },
        ignore_index=True
    )
    
    print('*'*100)
    print(f'{var_objetivo}')
    print('-'*100)
    print('Exactitud luego de búsqueda aleatoria en entrenamiento:', rf_random_best.score(X_train, y_train))
    print('Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag):', rf_random_best.oob_score_)
    print('Exactitud luego de búsqueda aleatoria en validación:', rf_random_best.score(X_test, y_test))
    print('*'*100)

Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.959731543624161
Exactitud del modelo inicial en validación: 0.9029411764705882
Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 'max_features': ['auto', 10, 11, 12],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

Si se probara todas las combinaciones se requeriría entrenar 1440 modelos
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Los hiperparámetros del mejor modelo son:
{'max_depth': 52,
 'max_features': 10,
 'min_samples_leaf': 1,
 'n_estimators': 1600}

****************************************************************************************************
FIBR_PREDS
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 1.0
E

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Los hiperparámetros del mejor modelo son:
{'max_depth': 220,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'n_estimators': 1400}

****************************************************************************************************
FIBR_JELUD
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 0.9968602825745683
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.9850863422291993
Exactitud luego de búsqueda aleatoria en validación: 0.9441176470588235
****************************************************************************************************
Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.9910226385636222
Exactitud del modelo inicial en validación: 0.9705882352941176
Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Los hiperparámetros del mejor modelo son:
{'max_depth': 199,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'n_estimators': 1000}

****************************************************************************************************
RAZRIV
----------------------------------------------------------------------------------------------------
Exactitud luego de búsqueda aleatoria en entrenamiento: 0.998829953198128
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.9894695787831513
Exactitud luego de búsqueda aleatoria en validación: 0.9705882352941176
****************************************************************************************************
Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.9838328075709779
Exactitud del modelo inicial en validación: 0.95
Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 'max_features': ['a

### 2.2) Ensamble - XGBoost

Entrenamos el modelo de XGBoost de tipo clasificación.

In [30]:
from xgboost import XGBRegressor
from pprint import pprint
from sklearn.model_selection import GridSearchCV
import time

## XGBoostRegressor
modelXGBR = XGBRegressor()


## Creamos la Grilla
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
learning_rate = [0.01,0.1]
max_depth = [i for i in range(2,8,2)]
start = time.time()

random_grid = {'max_depth': max_depth,
               'n_estimators': n_estimators,
               'colsample_bytree': [0.2, 0.6, 0.8],
               'min_child_weight': [3, 5, 7],
               'gamma': [0.3, 0.5, 0.7],
               'subsample': [0.4, 0.6, 0.8, 1],
               'learning_rate': learning_rate}

#### Búsqueda aleatoria de mejores hiperparámetros

In [31]:
xgb_random = RandomizedSearchCV(estimator = modelXGBR, 
                               param_distributions = random_grid, 
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2, 
                               random_state=0, 
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
xgb_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=...
                                          reg_alpha=None, reg_lambda=None,
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.2, 0.6, 0.8],
                                        'gamma': [0.3, 0.5, 0.7],
                                        'learning_rate': [0.01, 0.1],
                                        'max_depth': [2, 4, 6],
                                        'min_child_weight': [3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.4, 0.6, 0.8, 1]},
                   random_state=0, verbose=2)

In [32]:
## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
xgb_best_model = xgb_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(xgb_random.best_params_)
print()


score_XGBR_train = xgb_best_model.score(X_train, y_train)
score_XGBR_val = xgb_best_model.score(X_test, y_test)
print("*"*100)

resultados_modelo_acc_df=resultados_modelo_acc_df.append(
    {
        'Modelo' : 'XGBoost' , 
        'Accuracy Train' : score_XGBR_train,
        'Accuracy Test' : score_XGBR_val
    },
    ignore_index=True
)
print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_XGBR_train*100:.3f}%")
print(f"Exactitud luego de búsqueda en grilla en validación: {score_XGBR_val*100:.3f}%")
print("*"*100)
end = time.time()
print(f"{end-start} segundos")

Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.2,
 'gamma': 0.7,
 'learning_rate': 0.01,
 'max_depth': 4,
 'min_child_weight': 7,
 'n_estimators': 400,
 'subsample': 1}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 30.552%
Exactitud luego de búsqueda en grilla en validación: 5.303%
****************************************************************************************************
15.436229467391968 segundos


### 2.3) SVM

Entrenamos el modelo de SVM de tipo clasificación.

In [33]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

modelSVM = svm.SVC()

# Entrenamiento del modelo base - SVM
modelSVM.fit(X_train, y_train)
score_SVM_train = modelSVM.score(X_train, y_train)
score_SVM_test = modelSVM.score(X_test, y_test)


print(f"Exactitud con SVM base - Entrenamiento: {score_SVM_train}")
print(f"Exactitud con SVM base - Pruebas: {score_SVM_test}")

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Exactitud con SVM base - Entrenamiento: 0.4588235294117647
Exactitud con SVM base - Pruebas: 0.47352941176470587


#### Búsqueda aleatoria de mejores hiperparámetros

In [34]:
# Definición de grilla
random_grid = {'kernel': ['rbf'],
               'C': np.logspace(-4,4,9), # [0.0001, 0.001, ..., 10000]
               'gamma': np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]
              }

modelSVM_random = RandomizedSearchCV(estimator = modelSVM, 
                               param_distributions = random_grid, 
                               cv = 12,          # Validación cruzada 3-fold
                               verbose=2, 
                               random_state=0, 
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
modelSVM_random.fit(X_train, y_train)

## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
modelSVM_best_model = modelSVM_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(modelSVM_random.best_params_)
print()


score_SVM_train = modelSVM_best_model.score(X_train, y_train)
score_SVM_test = modelSVM_best_model.score(X_test, y_test)
print("*"*100)

resultados_modelo_acc_df=resultados_modelo_acc_df.append(
    {
        'Modelo' : 'SVM' ,
        'Accuracy Train' : score_SVM_train,
        'Accuracy Test' : score_SVM_test
    },
    ignore_index=True
)

print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_SVM_train:.3f}")
print(f"Exactitud luego de búsqueda en grilla en validación: {score_SVM_test:.3f}")

end = time.time()
print(end-start)

Fitting 12 folds for each of 10 candidates, totalling 120 fits


D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=12.
  warnings.warn(
D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Los hiperparámetros del mejor modelo son:
{'C': 0.01, 'gamma': 1.0, 'kernel': 'rbf'}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 0.444
Exactitud luego de búsqueda en grilla en validación: 0.474
97.53587985038757


### 2.4. Naive Bayes

Entrenamos el modelo de Naive Bayes (Gausiano) de tipo clasificación.

In [35]:
from sklearn.naive_bayes import GaussianNB

In [36]:
modelNB = GaussianNB()
#modelNB.fit(X_train, y_train)

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(params_NB)

modelNB_random = RandomizedSearchCV(estimator = modelNB, 
                               param_distributions=params_NB, 
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2, 
                               random_state=0, 
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
modelNB_random.fit(X_train, y_train)

## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
modelNB_best_model = modelNB_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(modelNB_random.best_params_)
print()


score_NB_train = modelNB_best_model.score(X_train, y_train)
score_NB_test = modelNB_best_model.score(X_test, y_test)
print("*"*100)

resultados_modelo_acc_df=resultados_modelo_acc_df.append(
    {
        'Modelo' : 'Naive Bayes - Gaussian' ,
        'Accuracy Train' : score_NB_train,
        'Accuracy Test' : score_NB_test
    },
    ignore_index=True
)

print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_NB_train:.3f}")
print(f"Exactitud luego de búsqueda en grilla en validación: {score_NB_test:.3f}")

end = time.time()

Los valores a probar en la búsqueda aleatoria son:
{'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.31012970e-02, 1.87381742e-02,
       1.51991108e-02, 1.23284674e-02, 1.00000000e-02, 8.11130831e-03,
       6.57933225e-03, 5.33669923e-03, 4.32876128e-03, 3.51119173e-03,
       2.84803587e-03, 2.31012970e-03, 1.87381742e-03, 1.51991108e-03,
       1.23284674e-03, 1.00000000e-03, 8.11130831e-04, 6.57933225e-04,
       5.33669923e-04, 4.32876128e-04, 3.51119173e-04, 2.84803587e-04,
       2.31012970e-04, 1.87381742e-04, 1.51991108e-04, 1.23284674e-04,
       1.00000000e-04, 8.11130831e-05, 6.57933225e-05, 5.33669923e-05,
       4.32876128e-05, 3.51119173e-05, 2.84803587e-05, 2.31012970e-05,
       1

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Los hiperparámetros del mejor modelo son:
{'var_smoothing': 0.657933224657568}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 0.388
Exactitud luego de búsqueda en grilla en validación: 0.282


In [37]:
resultados_modelo_acc_df

,Modelo,Accuracy Train,Accuracy Test
0,RandomForest,0.937500,0.491176
1,XGBoost,0.305517,0.053027
2,SVM,0.444118,0.473529
3,Naive Bayes - Gaussian,0.388235,0.282353
